# Automatic Intrisinsics Calibration

Intrinsics calibration for a camera 


Action plan:
* Generate first model of intrinsics calibration using an existing model
* Manual extrinsics calibration using existing models
* Generate baseline velocity estimation

Candidate models:
* https://github.com/alexvbogdan/DeepCalib
* https://arxiv.org/abs/1909.03459

The Blind distortion correction model did not perform well at all. The results essentially generated results that made the input images slightly blurrier. The main advantage of the model was just that it uses the same version of Cuda and torch as our model. 

## Deep Calib

Deep calib appears to give much more promising results. The results are based on the pretrained models that are provided in the source of the project. The models that were chosen are based on 

Here is a reference to the paper:
https://drive.google.com/file/d/1pZgR3wNS6Mvb87W0ixOHmEVV6tcI8d50/view

The convenient thing about Deep Calib is that it generates the intrinsic parameters for the camera directly. The output of the model is the focal length and distortion coeffient for the camera which makes it possible to have the rectification step part of the perception pipeline without the need for another expensive neural network to be applied to every image before detection.

Further validation is needed to get an accurate understanding of the performance of deepcalib on the dataset. Ground truth will be needed to get metrics of perforamnce.

# Extrinsics Calibration

Extrinsics calibration is the problem of finding the position and orientation of a camera in space. In the context of this task, the extrinsics calibration procedure requires being able to estimate the roll, pitch and yaw of a camera of a particular scene as well as its position in space (x, y, z). Traditionally extrinsics calibration requires estimating each of these parameters which yeilds 6 DOF for the model. Fortunately, we don't really care about all the information from the environment and what we really care about is just the height of the camera (z), as well as the camera's pitch, and roll with respect to the ground plane. 

The ground plane would be defined the normal vector z pointing straight up (the vector $[0\; 0 \; 1]$) so the equation of the plane with the origin lying directly below the camera would be:

$n^T \cdot (x-p_{origin}) = 0$

Since the origin is also on the plane, the equation simplifies to:
$n^T \cdot x = 0$. 
Which just says that any point with z component 0 lies on the plane since that's the only way that the dot product would be zero.

Below is an example of the reference frame for the camera: 


![thing](https://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/OWENS/LECT9/img4.gif
)    

Given this reference frame, you can generate the rigid transform from the optical center of the camera to the ground plane. 